In [54]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import time
from torch.utils import data
%matplotlib inline
import torchvision.transforms as standard_transforms
from torch.utils.data import DataLoader
import random
from torch import optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from PIL import Image, ImageOps, ImageFilter
import numbers
import pdb
import pandas as pd
import torch.nn.functional as F

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


In [56]:
!unzip ShanghaiTech_Crowd_Counting_Dataset.zip

unzip:  cannot find or open ShanghaiTech_Crowd_Counting_Dataset.zip, ShanghaiTech_Crowd_Counting_Dataset.zip.zip or ShanghaiTech_Crowd_Counting_Dataset.zip.ZIP.


In [57]:
class SHHA(data.Dataset):
    def __init__(self, data_path, mode, main_transform=None, img_transform=None, gt_transform=None):
        self.img_path = data_path + '/img'
        self.gt_path = data_path + '/den'
        self.data_files = [filename for filename in os.listdir(self.img_path) \
                           if os.path.isfile(os.path.join(self.img_path,filename))]
        self.num_samples = len(self.data_files) 
        self.main_transform=main_transform  
        self.img_transform = img_transform
        self.gt_transform = gt_transform     
    
    def __getitem__(self, index):
        fname = self.data_files[index]
        img, den = self.read_image_and_gt(fname)      
        if self.main_transform is not None:
            img, den = self.main_transform(img,den) 
        if self.img_transform is not None:
            img = self.img_transform(img)         
        if self.gt_transform is not None:
            den = self.gt_transform(den)               
        return img, den

    def __len__(self):
        return self.num_samples

    def read_image_and_gt(self,fname):
        img = Image.open(os.path.join(self.img_path,fname))
        if img.mode == 'L':
            img = img.convert('RGB')

        den = pd.read_csv(os.path.join(self.gt_path,os.path.splitext(fname)[0] + '.csv'), sep=',',header=None).values
        
        den = den.astype(np.float32, copy=False)    
        den = Image.fromarray(den)  
        return img, den    

    def get_num_samples(self):
        return self.num_samples

In [58]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.cur_val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, cur_val):
        self.cur_val = cur_val
        self.sum += cur_val
        self.count += 1
        self.avg = self.sum / self.count


In [59]:
# ===============================img tranforms============================

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, mask, bbx=None):
        if bbx is None:
            for t in self.transforms:
                img, mask = t(img, mask)
            return img, mask
        for t in self.transforms:
            img, mask, bbx = t(img, mask, bbx)
        return img, mask, bbx

class RandomHorizontallyFlip(object):
    def __call__(self, img, mask, bbx=None):
        if random.random() < 0.5:
            if bbx is None:
                return img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT)
            w, h = img.size
            xmin = w - bbx[:,3]
            xmax = w - bbx[:,1]
            bbx[:,1] = xmin
            bbx[:,3] = xmax
            return img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT), bbx
        if bbx is None:
            return img, mask
        return img, mask, bbx

class RandomCrop(object):
    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size
        self.padding = padding

    def __call__(self, img, mask, dst_size=None):
        if self.padding > 0:
            img = ImageOps.expand(img, border=self.padding, fill=0)
            mask = ImageOps.expand(mask, border=self.padding, fill=0)

        assert img.size == mask.size
        w, h = img.size
        if dst_size is None:
            th, tw = self.size
        else:
            th, tw = dst_size
        if w == tw and h == th:
            return img, mask
        if w < tw or h < th:
            return img.resize((tw, th), Image.BILINEAR), mask.resize((tw, th), Image.NEAREST)

        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        return img.crop((x1, y1, x1 + tw, y1 + th)), mask.crop((x1, y1, x1 + tw, y1 + th))


class CenterCrop(object):
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, img, mask):
        w, h = img.size
        th, tw = self.size
        x1 = int(round((w - tw) / 2.))
        y1 = int(round((h - th) / 2.))
        return img.crop((x1, y1, x1 + tw, y1 + th)), mask.crop((x1, y1, x1 + tw, y1 + th))


class Scale(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, img, mask):
        if img.size != mask.size:
            print( img.size )
            print( mask.size )          
        assert img.size == mask.size
        w, h = img.size
        if (w <= h and w == self.size) or (h <= w and h == self.size):
            return img, mask
        if w < h:
            ow = self.size
            oh = int(self.size * h / w)
            return img.resize((ow, oh), Image.BILINEAR), mask.resize((ow, oh), Image.NEAREST)
        else:
            oh = self.size
            ow = int(self.size * w / h)
            return img.resize((ow, oh), Image.BILINEAR), mask.resize((ow, oh), Image.NEAREST)


# ===============================label tranforms============================

class DeNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor


class LabelNormalize(object):
    def __init__(self, para):
        self.para = para

    def __call__(self, tensor):
        # tensor = 1./(tensor+self.para).log()
        tensor = torch.from_numpy(np.array(tensor))
        tensor = tensor*self.para
        return tensor

    
class GTScaleDown(object):
    def __init__(self, factor=8):
        self.factor = factor

    def __call__(self, img):
        w, h = img.size
        if self.factor==1:
            return img
        tmp = np.array(img.resize((w//self.factor, h//self.factor), Image.BICUBIC))*self.factor*self.factor
        img = Image.fromarray(tmp)
        return img

In [60]:
TRAIN_SIZE = (576,768)
LABEL_FACTOR = 1
TRAIN_BATCH_SIZE = 1
NUM_WORKERS = 0

def get_min_size(batch):

    min_ht = TRAIN_SIZE[0]
    min_wd = TRAIN_SIZE[1]

    for i_sample in batch:
        
        _,ht,wd = i_sample.shape
        if ht<min_ht:
            min_ht = ht
        if wd<min_wd:
            min_wd = wd
    return min_ht,min_wd

def random_crop(img,den,dst_size):
    # dst_size: ht, wd

    _,ts_hd,ts_wd = img.shape

    x1 = random.randint(0, ts_wd - dst_size[1])//LABEL_FACTOR*LABEL_FACTOR
    y1 = random.randint(0, ts_hd - dst_size[0])//LABEL_FACTOR*LABEL_FACTOR
    x2 = x1 + dst_size[1]
    y2 = y1 + dst_size[0]

    label_x1 = x1//LABEL_FACTOR
    label_y1 = y1//LABEL_FACTOR
    label_x2 = x2//LABEL_FACTOR
    label_y2 = y2//LABEL_FACTOR

    return img[:,y1:y2,x1:x2], den[label_y1:label_y2,label_x1:label_x2]


def SHHA_collate(batch):
    # @GJY 
    r"""Puts each data field into a tensor with outer dimension batch size"""

    transposed = list(zip(*batch)) # imgs and dens
    imgs, dens = [transposed[0],transposed[1]]


    error_msg = "batch must contain tensors; found {}"
    if isinstance(imgs[0], torch.Tensor) and isinstance(dens[0], torch.Tensor):
        
        min_ht, min_wd = get_min_size(imgs)

        # print min_ht, min_wd

        # pdb.set_trace()
        
        cropped_imgs = []
        cropped_dens = []
        for i_sample in range(len(batch)):
            _img, _den = random_crop(imgs[i_sample],dens[i_sample],[min_ht,min_wd])
            cropped_imgs.append(_img)
            cropped_dens.append(_den)


        cropped_imgs = torch.stack(cropped_imgs, 0, out=share_memory(cropped_imgs))
        cropped_dens = torch.stack(cropped_dens, 0, out=share_memory(cropped_dens))

        return [cropped_imgs,cropped_dens]

    raise TypeError((error_msg.format(type(batch[0]))))


def loading_data():
    mean_std = ([0.410824894905, 0.370634973049, 0.359682112932], [0.278580576181, 0.26925137639, 0.27156367898])
    log_para = 100.
    factor = 1
    TRAIN_BATCH_SIZE = 1
    DATA_PATH = "ProcessedData/shanghaitech_part_A"
    VAL_BATCH_SIZE = 3
    
    
    train_main_transform = Compose([
        RandomHorizontallyFlip()
    ])
    img_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
    gt_transform = standard_transforms.Compose([
        GTScaleDown(factor),
        LabelNormalize(log_para)
    ])
    restore_transform = standard_transforms.Compose([
        DeNormalize(*mean_std),
        standard_transforms.ToPILImage()
    ])

    train_set = SHHA(DATA_PATH+'/train', 'train',main_transform=train_main_transform, img_transform=img_transform, gt_transform=gt_transform)
    train_loader =None
    if TRAIN_BATCH_SIZE==1:
        train_loader = DataLoader(train_set, batch_size=1, shuffle=True, drop_last=True)
    elif TRAIN_BATCH_SIZE>1:
        train_loader = DataLoader(train_set, batch_size=TRAIN_BATCH_SIZE, num_workers=NUM_WORKERS, collate_fn=SHHA_collate, shuffle=True, drop_last=True)
    
    

    val_set = SHHA(DATA_PATH+'/test', 'test', main_transform=None, img_transform=img_transform, gt_transform=gt_transform)
    val_loader = DataLoader(val_set, batch_size=VAL_BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True, drop_last=False)

    return train_loader, val_loader, restore_transform

In [61]:
train_loader, val_loader, restore_transform = loading_data()

In [62]:
class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            self.frontend.load_state_dict(mod.features[0:23].state_dict())
            
    def forward(self,x):
        size = x.size()
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        x = F.upsample(x, size = size[2:])
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)  

In [63]:
GPU_ID = [0]
seed = 1

In [64]:
now = time.strftime("%m-%d_%H-%M", time.localtime())
EXP_NAME = now + '_' + "SHHA" + "_" + "CSRNet"
EXP_PATH = './exp'
net = 'CSRNet'
LR = 1e-5 # learning rate
LR_DECAY = 0.995 # decay rate
LR_DECAY_START = -1 # when training epoch is more than it, the learning rate will be begin to decay
NUM_EPOCH_LR_DECAY = 1 # decay frequency
MAX_EPOCH = 200
PRINT_FREQ = 1
LOG_PARA = 100.


class Trainer():
    def __init__(self, dataloader):

        self.data_mode = 'SHHA'
        self.exp_name = EXP_NAME
        self.exp_path = EXP_PATH
        self.LOG_PARA = LOG_PARA

        self.net_name = net
        self.net = CSRNet().to(device)
        self.loss = nn.MSELoss().to(device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=LR, weight_decay=1e-4)
        # self.optimizer = optim.SGD(self.net.parameters(), cfg.LR, momentum=0.95,weight_decay=5e-4)
        self.scheduler = StepLR(self.optimizer, step_size=NUM_EPOCH_LR_DECAY, gamma=LR_DECAY)          

        self.train_record = {'best_mae': 1e20, 'best_mse':1e20, 'best_model_name': ''}

        self.epoch = 0
        self.i_tb = 0
        
        """
        if cfg.PRE_GCC:
            self.net.load_state_dict(torch.load(cfg.PRE_GCC_MODEL))
        """
        
        self.train_loader, self.val_loader, self.restore_transform = dataloader()



    def forward(self):

        # self.validate_V3()
        for epoch in range(self.epoch, MAX_EPOCH):
            self.epoch = epoch
            if epoch > LR_DECAY_START:
                self.scheduler.step()
                
            # training    
            self.train()

            # validation
            if epoch%VAL_FREQ==0 or epoch>VAL_DENSE_START:
                self.validate_V1()
                torch.save(self.net.state_dict(), "my_model_weigths.pth")


    def train(self): # training for all datasets
        self.net.train()
        
        for i, data in enumerate(self.train_loader, 0):
            img, gt_map = data
            img = Variable(img).to(device)
            gt_map = Variable(gt_map).to(device)

            self.optimizer.zero_grad()
            # pred_map = self.net(img, gt_map)
            
            pred_density_map = self.net(img) 
            loss = self.loss(pred_density_map, gt_map)
            loss.backward()
            self.optimizer.step()

            if (i + 1) % PRINT_FREQ == 0:
                self.i_tb += 1
                # self.writer.add_scalar('train_loss', loss.item(), self.i_tb)
                print( '[ep %d][it %d][loss %.4f][lr %.4f]' % \
                        (self.epoch + 1, i + 1, loss.item(), self.optimizer.param_groups[0]['lr']*10000) )
                print( '        [cnt: gt: %.1f pred: %.2f]' % (gt_map[0].sum().data/self.LOG_PARA, pred_density_map[0].sum().data/self.LOG_PARA) )   


    def validate_V1(self):# validate_V1 for SHHA, SHHB, UCF-QNRF, UCF50

        self.net.eval()
        
        losses = AverageMeter()
        maes = AverageMeter()
        mses = AverageMeter()

        for vi, data in enumerate(self.val_loader, 0):
            img, gt_map = data

            with torch.no_grad():
                img = Variable(img).to(device)
                gt_map = Variable(gt_map).to(device)

                pred_map = self.net.forward(img,gt_map)

                pred_map = pred_map.data.cpu().numpy()
                gt_map = gt_map.data.cpu().numpy()

                for i_img in range(pred_map.shape[0]):
                
                    pred_cnt = np.sum(pred_map[i_img])/self.LOG_PARA
                    gt_count = np.sum(gt_map[i_img])/self.LOG_PARA

                    
                    losses.update(self.net.loss.item())
                    maes.update(abs(gt_count-pred_cnt))
                    mses.update((gt_count-pred_cnt)*(gt_count-pred_cnt))
            
        mae = maes.avg
        mse = np.sqrt(mses.avg)
        loss = losses.avg

        self.writer.add_scalar('val_loss', loss, self.epoch + 1)
        self.writer.add_scalar('mae', mae, self.epoch + 1)
        self.writer.add_scalar('mse', mse, self.epoch + 1)

In [65]:
"""
if seed is not None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    
gpus = GPU_ID
if len(gpus)==1:
    torch.cuda.set_device(gpus[0])
"""

torch.backends.cudnn.benchmark = True

#------------Start Training------------
cc_trainer = Trainer(loading_data)
cc_trainer.forward()

/home/simon/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 683, 1024])) that is different to the input s

[ep 1][it 1][loss 0.0516][lr 0.0995]
        [cnt: gt: 401.6 pred: -2.95]


/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 768, 1023])) that is different to the input size (torch.Size([1, 1, 768, 1023])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 